In [1]:
import os
import pandas as pd
import numpy as np

import pyreadr

#import dask_mpi as dm
from dask.distributed import Client, progress
import dask

from datetime import datetime
from joblib import Parallel, delayed
from sklearn.ensemble import RandomForestRegressor
from joblib import dump, load
import joblib

In [2]:
!pwd

'pwd' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
DP02_location = "/Users/lichao/Library/CloudStorage/OneDrive-KyushuUniversity/02_Article/03_RStudio/"
DP02_result_location = "/Users/lichao/Library/CloudStorage/OneDrive-KyushuUniversity/02_Article/03_RStudio/08_PyResults/"

In [2]:
DP02_location = "D:/OneDrive - Kyushu University/02_Article/03_RStudio/"
DP02_result_location = "D:/OneDrive - Kyushu University/02_Article/03_RStudio/08_PyResults/"

In [30]:
client = Client(n_workers=3, nthreads=1)

D:\Anaconda3\envs\Usegdal\lib\site-packages\distributed\node.py:177: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 62132 instead
  warnings.warn(


In [31]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:62132/status,
Dashboard: http://127.0.0.1:62132/status,Workers: 3
Total threads: 12,Total memory: 63.92 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:62133,Workers: 3
Dashboard: http://127.0.0.1:62132/status,Total threads: 12
Started: Just now,Total memory: 63.92 GiB
Comm: tcp://127.0.0.1:62157,Total threads: 4
Dashboard: http://127.0.0.1:62159/status,Memory: 21.31 GiB
Nanny: tcp://127.0.0.1:62138,


In [5]:
dataset = pyreadr.read_r(DP02_location + "02_Data/SP_Data_49Variable_Weights_changeRangeOfLandCover_RdsVer.Rds")
dataset = dataset[None]
y = np.array(dataset.iloc[:, 0:1].values.flatten(), dtype='float64')
X = np.array(dataset.iloc[:, 1:50], dtype='float64')

In [6]:
model = RandomForestRegressor(n_estimators=1000, oob_score=True, 
                               random_state=1, max_features = 9, n_jobs=-1)
with joblib.parallel_backend("dask"): model.fit(X, y)

In [7]:
# SHAP
import dalex as dx
model_rf_exp = dx.Explainer(model, X, y, label = "RF Pipeline")

Preparation of a new explainer is initiated

  -> data              : numpy.ndarray converted to pandas.DataFrame. Columns are set as string numbers.
  -> data              : 89273 rows 49 cols
  -> target variable   : 89273 values
  -> model_class       : sklearn.ensemble._forest.RandomForestRegressor (default)
  -> label             : RF Pipeline
  -> predict function  : <function yhat_default at 0x000001C722A12680> will be used (default)
  -> predict function  : Accepts pandas.DataFrame and numpy.ndarray.


D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


  -> predicted values  : min = 1.18, mean = 24.3, max = 36.0
  -> model type        : regression will be used (default)
  -> residual function : difference between y and yhat (default)


D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


  -> residuals         : min = -11.2, mean = 0.0166, max = 7.13
  -> model_info        : package sklearn

A new explainer has been created!


In [32]:
X_scattered = client.scatter(X)
model_rf_exp_scattered = client.scatter(model_rf_exp)

In [33]:
def singleSHAPprocess(obs_num, X, model_rf_exp):
    test_obs = X[obs_num:obs_num+1,:]
    shap_test = model_rf_exp.predict_parts(test_obs, type = 'shap', 
                                           B = 5, N = 100)
    result = shap_test.result[shap_test.result.B == 0]
    result = result[['contribution', 'variable_name']]
    result = result.transpose()
    result = result.rename(columns=result.iloc[1])
    result = result.drop(['variable_name'], axis=0)
    result = result.reset_index(drop=True)
    return result

In [36]:
%%time
results = []
for obs_num in list(range(100)):
    results.append(dask.delayed(singleSHAPprocess)(obs_num, X_scattered, model_rf_exp_scattered))

Wall time: 5 ms


In [ ]:
%%time
test = dask.compute(*results) 

In [29]:
%%time
test[0]

Wall time: 0 ns


,12,18,31,14,10,30,23,47,1,0,...,27,35,40,7,6,34,19,8,9,4
0,1.210959,-1.086528,-0.936787,-0.880058,-0.863376,0.812714,-0.287092,0.229171,0.165282,0.161684,...,0.007528,0.007021,-0.005025,0.004678,-0.003876,0.003344,0.002366,0.001918,0.001248,0.0


In [22]:
results

[Delayed('singleSHAPprocess-9412b495-6588-477a-9228-8f15d5f90c3f'),
 Delayed('singleSHAPprocess-77d44bc5-2977-48e4-a3b8-11c2caeb3087'),
 Delayed('singleSHAPprocess-92b82389-fa2c-4fc5-82a0-5dcca792b82e'),
 Delayed('singleSHAPprocess-f74e49f7-9bfa-42d2-a6a7-e065161c8919'),
 Delayed('singleSHAPprocess-113ad494-fc7a-48db-a5e8-44b39aa65c51'),
 Delayed('singleSHAPprocess-932642f0-c065-4692-9468-83d750a776b7'),
 Delayed('singleSHAPprocess-cf72239b-fdad-4ba3-a1ff-17c77b39175f'),
 Delayed('singleSHAPprocess-cbb51a8d-dc7a-4e22-b68c-6d611644fb73'),
 Delayed('singleSHAPprocess-914b7395-226c-4439-887e-2bb171cdf1f7'),
 Delayed('singleSHAPprocess-27ca34bd-cd5a-4ac1-947e-7b6aab4f7af6'),
 Delayed('singleSHAPprocess-a33e8041-aa48-45f4-a903-7d906a88d0ed'),
 Delayed('singleSHAPprocess-a107f4c6-5490-4826-bc8b-70601b4f0994'),
 Delayed('singleSHAPprocess-14f6db0a-1896-4b59-87e4-476611c5b768'),
 Delayed('singleSHAPprocess-4dc1cc9f-aad6-4f8c-acb7-281698747d11'),
 Delayed('singleSHAPprocess-9856a034-45b4-4ddc-a

In [71]:
list(range(8))

[0, 1, 2, 3, 4, 5, 6, 7]

In [23]:
client.close()

In [25]:
%%time
results_bag = joblib.Parallel(n_jobs=3, verbose=10000, backend = 'threading')(
    joblib.delayed(singleSHAPprocess)(int(obs_num), X, model_rf_exp)
    for obs_num in list(range(20)))

[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.


D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor 

D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor 

D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor 

D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor 

...

D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor 

D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor 

D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor 

D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor 

D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor 